In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/brain-mri-images-for-brain-tumor-detection/no/34 no.jpg
/kaggle/input/brain-mri-images-for-brain-tumor-detection/no/N20.JPG
/kaggle/input/brain-mri-images-for-brain-tumor-detection/no/N1.JPG
/kaggle/input/brain-mri-images-for-brain-tumor-detection/no/49 no.jpg
/kaggle/input/brain-mri-images-for-brain-tumor-detection/no/N15.jpg
/kaggle/input/brain-mri-images-for-brain-tumor-detection/no/No18.jpg
/kaggle/input/brain-mri-images-for-brain-tumor-detection/no/31 no.jpg
/kaggle/input/brain-mri-images-for-brain-tumor-detection/no/no 6.jpg
/kaggle/input/brain-mri-images-for-brain-tumor-detection/no/3 no.jpg
/kaggle/input/brain-mri-images-for-brain-tumor-detection/no/no 5.jpeg
/kaggle/input/brain-mri-images-for-brain-tumor-detection/no/no 1.jpg
/kaggle/input/brain-mri-images-for-brain-tumor-detection/no/26 no.jpg
/kaggle/input/brain-mri-images-for-brain-tumor-detection/no/N26.JPG
/kaggle/input/brain-mri-images-for-brain-tumor-detection/no/N5.jpg
/kaggle/input/brain-mri-images-for-b

In [2]:
import cv2
import numpy as np
import skimage.feature as ft
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical


In [3]:
# Load and Preprocess Image
def load_and_preprocess_image(image_path, target_size=(128, 128)):
    image = cv2.imread(image_path)
    image = cv2.resize(image, target_size)
    image = cv2.GaussianBlur(image, (5, 5), 0)  # Noise removal
    image = image / 255.0  # Normalization
    return image

In [4]:
# Edge Detection
def edge_detection(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return cv2.Canny(gray, 50, 150)
    

In [5]:
# GLCM Texture Features
def compute_glcm_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    glcm = ft.greycomatrix(gray, [1], [0], 256, symmetric=True, normed=True)
    features = [
        ft.greycoprops(glcm, 'contrast')[0, 0],
        ft.greycoprops(glcm, 'dissimilarity')[0, 0],
        ft.greycoprops(glcm, 'homogeneity')[0, 0],
        ft.greycoprops(glcm, 'energy')[0, 0],
        ft.greycoprops(glcm, 'correlation')[0, 0]
    ]
    return features

In [6]:
# Otsu Thresholding
def otsu_thresholding(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return thresh

In [7]:
# Load dataset with separate folders for each class
def load_dataset(folder, labels):
    # labels = {"glioma": 0, "meningioma": 1, "pituitary tumor": 2}
    images = []
    image_labels = []
    
    for label, value in labels.items():
        class_folder = os.path.join(folder, label)
        for filename in os.listdir(class_folder):
            img_path = os.path.join(class_folder, filename)
            img = load_and_preprocess_image(img_path)
            images.append(img)
            image_labels.append(value)
    
    return np.array(images), np.array(image_labels)

In [8]:



# Load dataset
figshare_folder = "/kaggle/input/figshare-brain-tumor-classification/enhance_data_adaptive_clahe/enhance_data"
mri_folder = "/kaggle/input/brain-mri-images-for-brain-tumor-detection"
figshare_labels = {"glioma": 0, "meningioma": 1, "pituitary tumor": 2}
mri_label = {"no": 3}

X_figshare, y_figshare = load_dataset(figshare_folder, figshare_labels)
X_mri, y_mri = load_dataset(mri_folder, mri_label)

# Merge datasets
X_combined = np.concatenate((X_figshare, X_mri), axis=0)
y_combined = np.concatenate((y_figshare, y_mri), axis=0)
y_combined = to_categorical(y_combined, num_classes=4)  # Convert to one-hot encoding

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X_combined, y_combined, test_size=0.2, random_state=42)






In [16]:
# from tensorflow.keras.layers import BatchNormalization
def build_cnn_model(input_shape=(128, 128, 3), num_classes=4):
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        MaxPooling2D(2,2),
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        # Conv2D(64, (8, 8), activation = 'relu'),
        # MaxPooling2D(2,2),
        Conv2D(128, (8,8), activation = 'relu'),
        MaxPooling2D(2,2),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(num_classes, activation='softmax')
        
        # Conv2D(32, (3,3), activation='relu'),
        # BatchNormalization()
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model



# Train CNN for tumor type classification
cnn_model = build_cnn_model()
cnn_model.fit(np.array(X_train).reshape(-1, 128, 128, 3), y_train, epochs=10, batch_size=32, validation_data=(np.array(X_test).reshape(-1, 128, 128, 3), y_test))


Epoch 1/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 82s 995ms/step - accuracy: 0.4486 - loss: 1.3103 - val_accuracy: 0.5940 - val_loss: 0.9377
Epoch 2/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 79s 980ms/step - accuracy: 0.6764 - loss: 0.7998 - val_accuracy: 0.6825 - val_loss: 0.7494
Epoch 3/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 79s 991ms/step - accuracy: 0.7588 - loss: 0.5749 - val_accuracy: 0.7725 - val_loss: 0.5436
Epoch 4/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 79s 986ms/step - accuracy: 0.8184 - loss: 0.4391 - val_accuracy: 0.7757 - val_loss: 0.4818
Epoch 5/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 79s 990ms/step - accuracy: 0.8648 - loss: 0.3556 - val_accuracy: 0.8310 - val_loss: 0.3723
Epoch 6/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 82s 993ms/step - accuracy: 0.8744 - loss: 0.3035 - val_accuracy: 0.8752 - val_loss: 0.2928
Epoch 7/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 79s 983ms/step - accuracy: 0.9112 - loss: 0.2270 - val_accuracy: 0.8626 - val_loss: 0.3149
Epoch 8/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 79s 992ms/step - accuracy: 0.9351 - loss: 0.1743 - val_accu

In [12]:
test_loss, test_acc = cnn_model.evaluate(X_test, y_test)
print(f"Model Accuracy: {test_acc * 100:.2f}%")


20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 222ms/step - accuracy: 0.9115 - loss: 0.2586
Model Accuracy: 90.36%


In [18]:

# Final Prediction Pipeline
def predict_tumor(image_path):
    image = load_and_preprocess_image(image_path)
    image = image.reshape(1, 128, 128, 3)
    tumor_type = np.argmax(cnn_model.predict(image))
    classes = ["Glioma", "Meningioma", "Pituitary", "No Tumor"]
    return f"Prediction: {classes[tumor_type]}"

In [19]:
print(predict_tumor('/kaggle/input/brain-mri-images-for-brain-tumor-detection/no/15 no.jpg'))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
Prediction: No Tumor
